In [1]:
from scipy import stats as st
import os
import json
from appdirs import user_data_dir
from varname import nameof
from datetime import datetime
import numpy as np
from scipy import stats as st

file_name = 'exp_log'
output_name = 'exp_kpi'
time_format = "%H:%M:%S.%f"

def get_log_path(file_name, format="txt"):
    return os.path.join(user_data_dir('openvisualizer'), file_name+"."+format)

def openfile(func):
    def inner(inputfile):
        with open(inputfile, 'r') as f:
            return func(f)
    return inner

def parse_time(time_s):
    return datetime.strptime(time_s, time_format)

def generate_stats(values):
    return {
        "min": np.min(values),
        "max": np.max(values),
        "mean": np.mean(values),
        "std": np.std(values),
        "75%": np.percentile(values, 75),
        "99%": np.percentile(values, 99)
    }

In [3]:
@openfile
def kpis_all(inputfile):
    allstats = {
        "start_root": None,
        "motes": {}
    }

    for line in inputfile:
        logline = json.loads(line)
        time = logline['time']
        time = parse_time(time)
        state = logline['0_state']

        mote_id = None
        if "mote_id" in logline:
            mote_id = logline['mote_id']

        if state == 'root' and allstats["start_root"] is None:
            allstats["start_root"] = time
        
        if state == 'join' and allstats["start_root"]:
            join_at = time
            join_time = max((join_at - allstats["start_root"]).total_seconds() * 1000, 0)
                
            if mote_id not in allstats['motes']:
                allstats['motes'][mote_id] = {'mote_join_time': None}
            else:
                if allstats['motes'][mote_id]["mote_join_time"]: continue            
            
            allstats['motes'][mote_id]["mote_join_time"] = join_time
        
        if state == 'trickle':
            Nstate = logline['Nstates']

            # pkt pf
            if "trickle" not in allstats['motes'][mote_id]:
                allstats['motes'][mote_id]["trickle"] = {}

            if Nstate not in allstats['motes'][mote_id]["trickle"]:
                allstats['motes'][mote_id]["trickle"][Nstate] = {}
            
            temp = logline
            del temp['0_state']
            del temp['Nstates']
            del temp['mote_id']
            del temp['time']

            for key in logline.keys():
                allstats['motes'][mote_id]["trickle"][Nstate][key] = logline[key]


        if state == 'send':
            counter = logline["counter"]

            # keep update to get the last
            allstats['motes'][mote_id]["mote_ambr"] = logline["ambr"]
            
            if mote_id not in allstats['motes']:
                allstats['motes'][mote_id] = {}
        
            # mote level 
            if "mote_total_send" not in allstats['motes'][mote_id]:
                allstats['motes'][mote_id]["mote_total_send"] = 0

            allstats['motes'][mote_id]["mote_total_send"] += 1
            
            if "mote_total_fail" not in allstats['motes'][mote_id]:
                allstats['motes'][mote_id]["mote_total_fail"] = 0
            
            if logline["is_failed"]:
                allstats['motes'][mote_id]["mote_total_fail"] += 1

            # pkt time
            if "packets" not in allstats['motes'][mote_id]:
                allstats['motes'][mote_id]["packets"] = {}

            if counter not in allstats['motes'][mote_id]["packets"]:
                allstats['motes'][mote_id]["packets"][counter] = {}
            
            allstats['motes'][mote_id]["packets"][counter]["send_time"] = time

        if state == 'receive':
            counter = logline["counter"]
            
            if mote_id not in allstats['motes']:
                allstats['motes'][mote_id] = {}
        
            # mote level 
            if "mote_total_receive" not in allstats['motes'][mote_id]:
                allstats['motes'][mote_id]["mote_total_receive"] = 0

            allstats['motes'][mote_id]["mote_total_receive"] += 1

            # pkt time
            if "packets" not in allstats['motes'][mote_id]:
                allstats['motes'][mote_id]["packets"] = {}

            if counter not in allstats['motes'][mote_id]["packets"]:
                allstats['motes'][mote_id]["packets"][counter] = {}
            
            allstats['motes'][mote_id]["packets"][counter]["receive_time"] = time
        
        if state == 'receive' or state == 'send':
            counter = logline["counter"]

            if (
                'send_time' in allstats['motes'][mote_id]["packets"][counter] and
                'receive_time' in allstats['motes'][mote_id]["packets"][counter]
            ):
                send_ = allstats['motes'][mote_id]["packets"][counter]['send_time']
                receive_ = allstats['motes'][mote_id]["packets"][counter]['receive_time']

                diff = max((receive_ - send_).total_seconds() * 1000, 0)
                allstats['motes'][mote_id]["packets"][counter]["pkt_latency"] = diff

                del allstats['motes'][mote_id]["packets"][counter]['send_time']
                del allstats['motes'][mote_id]["packets"][counter]['receive_time']

    del allstats['start_root']
    all_motes_data = {}

    for mote in allstats['motes'].keys():
        # calculate latency
        latencies = []
        for pkt in allstats['motes'][mote]['packets'].keys():
            pkt_latency = allstats['motes'][mote]['packets'][pkt]["pkt_latency"]
            latencies.append(pkt_latency)
        del allstats['motes'][mote]['packets']

        allstats['motes'][mote_id]["mote_min_latency"] = np.min(latencies)
        allstats['motes'][mote_id]["mote_max_latency"] = np.max(latencies)
        allstats['motes'][mote_id]["mote_avg_latency"] = np.mean(latencies)

        # calculate mote pdr
        pdr = allstats['motes'][mote_id]["mote_total_receive"] / allstats['motes'][mote_id]["mote_total_send"]
        allstats['motes'][mote_id]["mote_pdr"] = pdr

        # get ambr
        ambr = allstats['motes'][mote_id]["mote_ambr"]

        # gather mote data for global stats
        if 'latency' not in all_motes_data:
            all_motes_data['latency'] = []
        if 'pdr' not in all_motes_data:
            all_motes_data['pdr'] = []
        if 'ambr' not in all_motes_data:
            all_motes_data['ambr'] = []
            
        all_motes_data['latency'].extend(latencies)
        all_motes_data['pdr'].append(pdr)
        all_motes_data['ambr'].append(ambr)

        for trickle_state in allstats['motes'][mote]['trickle'].keys():
            for param in allstats['motes'][mote]['trickle'][trickle_state].keys():
                value = float(allstats['motes'][mote]['trickle'][trickle_state][param])

                key = "trickle_"+param
                if key not in all_motes_data:
                    all_motes_data[key] = []
                
                all_motes_data[key].append(value)

    # calculate mote latency stat
    for key in all_motes_data.keys():
        allstats["global_"+key] = generate_stats(all_motes_data[key])    
    return allstats

filepath = get_log_path(file_name)
kpis = kpis_all(filepath)

output = get_log_path(output_name, "json")
with open(output, 'w') as f:
    f.write(json.dumps(kpis, indent=4, sort_keys=True, default=str))